# Loading Text Dataset

`Linux` `Ascend` `GPU` `CPU` `Data Preparation` `Beginner` `Intermediate` `Expert`

[![](https://gitee.com/mindspore/docs/raw/r1.3/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.3/docs/mindspore/programming_guide/source_en/load_dataset_text.ipynb)&emsp;[![](https://gitee.com/mindspore/docs/raw/r1.3/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/r1.3/programming_guide/en/mindspore_load_dataset_text.ipynb)&emsp;[![](https://gitee.com/mindspore/docs/raw/r1.3/resource/_static/logo_modelarts.png)](https://authoring-modelarts-cnnorth4.huaweicloud.com/console/lab?share-url-b64=aHR0cHM6Ly9taW5kc3BvcmUtd2Vic2l0ZS5vYnMuY24tbm9ydGgtNC5teWh1YXdlaWNsb3VkLmNvbS9ub3RlYm9vay9yMS4zL3Byb2dyYW1taW5nX2d1aWRlL2VuL21pbmRzcG9yZV9sb2FkX2RhdGFzZXRfdGV4dC5pcHluYg==&imageid=65f636a0-56cf-49df-b941-7d2a07ba8c8c)

## Overview

The `mindspore.dataset` module provided by MindSpore enables users to customize their data fetching strategy from disk. At the same time, data processing and tokenization operators are applied to the data. Pipelined data processing produces a continuous flow of data to the training network, improving overall performance.

In addition, MindSpore supports data loading in distributed scenarios. Users can define the number of shards while loading. For more details, see [Loading the Dataset in Data Parallel Mode](https://www.mindspore.cn/docs/programming_guide/en/r1.3/distributed_training_ascend.html#loading-the-dataset-in-data-parallel-mode).

This tutorial briefly demonstrates how to load and process text data using MindSpore.

## Preparations

1. Prepare the following text data.

    Welcome to Beijing!

    北京欢迎您！

    我喜欢English!

2. Create the `tokenizer.txt` file, copy the text data to the file, and save the file under `./datasets` directory. The directory structure is as follow.

In [5]:
    import os

    if not os.path.exists('./datasets'):
        os.mkdir('./datasets')
    file_handle=open('./datasets/tokenizer.txt',mode='w')
    file_handle.write('Welcome to Beijing \n北京欢迎您！ \n我喜欢English! \n')
    file_handle.close()

In [6]:
    ! tree ./datasets

./datasets
├── MNIST_Data
│   ├── test
│   │   ├── t10k-images-idx3-ubyte
│   │   └── t10k-labels-idx1-ubyte
│   └── train
│       ├── train-images-idx3-ubyte
│       └── train-labels-idx1-ubyte
└── tokenizer.txt

3 directories, 5 files


3. Import the `mindspore.dataset` and `mindspore.dataset.text` modules.

In [7]:
    import mindspore.dataset as ds
    import mindspore.dataset.text as text

## Loading Dataset

MindSpore supports loading common datasets in the field of text processing that come in a variety of on-disk formats. Users can also implement custom dataset class to load customized data. For detailed loading methods of various datasets, please refer to the [Loading Dataset](https://www.mindspore.cn/docs/programming_guide/en/r1.3/dataset_loading.html) chapter in the Programming Guide.

The following tutorial demonstrates loading datasets using the `TextFileDataset` in the `mindspore.dataset` module.

1. Configure the dataset directory as follows and create a dataset object.

In [8]:
    DATA_FILE = "./datasets/tokenizer.txt"
    dataset = ds.TextFileDataset(DATA_FILE, shuffle=False)

2. Create an iterator then obtain data through the iterator.

In [9]:
    for data in dataset.create_dict_iterator(output_numpy=True):
        print(text.to_str(data['text']))

Welcome to Beijing 
北京欢迎您！ 
我喜欢English! 


## Processing Data

For the data processing operators currently supported by MindSpore and their detailed usage methods, please refer to the [Processing Data](https://www.mindspore.cn/docs/programming_guide/en/r1.3/pipeline.html) chapter in the Programming Guide

The following tutorial demonstrates how to construct a pipeline and perform operations such as `shuffle` and `RegexReplace` on the text dataset.

1. Shuffle the dataset.

In [10]:
    ds.config.set_seed(58)
    dataset = dataset.shuffle(buffer_size=3)

    for data in dataset.create_dict_iterator(output_numpy=True):
        print(text.to_str(data['text']))

我喜欢English! 
Welcome to Beijing 
北京欢迎您！ 


2. Perform `RegexReplace` on the dataset.

In [11]:
    replace_op1 = text.RegexReplace("Beijing", "Shanghai")
    replace_op2 = text.RegexReplace("北京", "上海")
    dataset = dataset.map(operations=replace_op1)
    dataset = dataset.map(operations=replace_op2)

    for data in dataset.create_dict_iterator(output_numpy=True):
        print(text.to_str(data['text']))

我喜欢English! 
Welcome to Shanghai 
上海欢迎您！ 


## Tokenization

For the data tokenization operators currently supported by MindSpore and their detailed usage methods, please refer to the [Tokenizer](https://www.mindspore.cn/docs/programming_guide/en/r1.3/tokenizer.html) chapter in the Programming Guide.

The following tutorial demonstrates how to use the `WhitespaceTokenizer` to tokenize words with space.

1. Create a `tokenizer`.

In [12]:
    tokenizer = text.WhitespaceTokenizer()

2. Apply the `tokenizer`.

In [13]:
    dataset = dataset.map(operations=tokenizer)

3. Create an iterator and obtain data through the iterator.

In [14]:
    for data in dataset.create_dict_iterator(output_numpy=True):
        print(text.to_str(data['text']).tolist())

['我喜欢English!']
['Welcome', 'to', 'Shanghai']
['上海欢迎您！']
